In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import time

In [56]:
options = Options()
options.add_argument('--headless')  # GUI 없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')

service = Service('/usr/local/bin/chromedriver')  # 드라이버 경로 지정 (필요 시)

### 오늘자 업데이트 항목 가져오기

In [37]:
url = "https://korean.visitkorea.or.kr/kfes/list/wntyFstvlList.do"
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

In [38]:
# 스크롤을 끝까지 반복
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 로딩 대기

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("✅ 스크롤 완료!")
        break
    last_height = new_height

✅ 스크롤 완료!


In [39]:
def get_festival_info():
    event_info = []

    for i in range(1, 1240):  # 충분히 큰 수로 반복
        try:
            name_el = driver.find_element("xpath", f'/html/body/main/div/section[3]/div/div/div[4]/div[2]/ul/li[{i}]/a/div[2]/strong')
            name = name_el.text.strip()

            link_el = driver.find_element("xpath", f'/html/body/main/div/section[3]/div/div/div[4]/div[2]/ul/li[{i}]/a')
            href = link_el.get_attribute("href")

            event_info.append({"name": name, "url": href})
            print(f"{i}: {name} - {href}")

            time.sleep(0.5)  # 과도한 요청 방지
            
        except Exception as e:
            print(f"{i}: 항목 없음 또는 종료")
            break

    driver.quit()
    return event_info

In [40]:
if __name__ == "__main__":
    festivals = get_festival_info()
    print(f"\n총 {len(festivals)}개 축제 수집 완료")

1: 단양 소백산 철쭉제 - https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=0adb8f34-4b47-4eee-99f6-f64dc6717c68&cntntsNm=%EB%8B%A8%EC%96%91%EC%86%8C%EB%B0%B1%EC%82%B0%EC%B2%A0%EC%AD%89%EC%A0%9C
2: 밀양아리랑대축제 - https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=23b8642c-8c00-4278-a76b-2f7bd579c97a&cntntsNm=%EB%B0%80%EC%96%91%EC%95%84%EB%A6%AC%EB%9E%91%EB%8C%80%EC%B6%95%EC%A0%9C
3: 원주용수골꽃양귀비축제 - https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=aa361709-4651-40f8-903d-9cf355c86454&cntntsNm=%EC%9B%90%EC%A3%BC%EC%9A%A9%EC%88%98%EA%B3%A8%EA%BD%83%EC%96%91%EA%B7%80%EB%B9%84%EC%B6%95%EC%A0%9C
4: 공공한옥 밤마실 - https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=44361fc4-8826-49b1-b609-4628709e3ea9&cntntsNm=%EA%B3%B5%EA%B3%B5%ED%95%9C%EC%98%A5%EB%B0%A4%EB%A7%88%EC%8B%A4
5: 울산대공원 장미축제 - https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=b2047505-5240-4abb-ba9a-9ee82c6a966a&cntntsNm=%EC%9A%B8%EC%82%B0

In [47]:
# csv 업데이트
import datetime
import pandas as pd

today = datetime.date.today().strftime("%Y-%m-%d")
df = pd.DataFrame(festivals)
file_name = f"/home/ys0660/25-1/Demo/data/festivals_{today}.csv"

df.to_csv(file_name, index=False, encoding="utf-8-sig")



In [61]:
driver.quit()

### 상세 내용 가져오기

In [50]:
df = pd.read_csv('/home/ys0660/25-1/Demo/data/festivals_2025-05-22.csv', encoding = "utf-8-sig")

In [51]:
df.head()

,name,url
0,단양 소백산 철쭉제,https://korean.visitkorea.or.kr/kfes/detail/fs...
1,밀양아리랑대축제,https://korean.visitkorea.or.kr/kfes/detail/fs...
2,원주용수골꽃양귀비축제,https://korean.visitkorea.or.kr/kfes/detail/fs...
3,공공한옥 밤마실,https://korean.visitkorea.or.kr/kfes/detail/fs...
4,울산대공원 장미축제,https://korean.visitkorea.or.kr/kfes/detail/fs...


In [64]:
options = Options()
options.add_argument('--headless')  # GUI 없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')

service = Service('/usr/local/bin/chromedriver')  # 드라이버 경로 지정 (필요 시)

driver = webdriver.Chrome(service=service, options=options)

In [79]:
def get_text_or_none(xpath):
    try:
        return driver.find_element("xpath", xpath).text.strip()
    except:
        return None

def get_attr_or_none(xpath, attr):
    try:
        return driver.find_element("xpath", xpath).get_attribute(attr)
    except:
        return None

In [87]:
# 상세정보 저장할 리스트
details = []

# 하나씩 상세 페이지 접근
for idx, row in df.iterrows():
    url = row["url"]
    try:
        driver.get(url)
        time.sleep(1.5)

        name = get_text_or_none('/html/body/main/div/div/section[1]/div/div/div[1]/h2')
        print(idx, name)
        location = get_text_or_none('/html/body/main/div/div/section[3]/div/div[2]/div/div[2]/ul/li[2]/p')

        date_text = get_text_or_none('/html/body/main/div/div/section[3]/div/div[2]/div/div[2]/ul/li[1]/p')
        try:
            start_date, end_date = date_text.split(" ~ ")
        except Exception:
            if date_text is None or date_text.strip() == "":
                start_date, end_date = None, None
            else:
                start_date, end_date = date_text, date_text

        description = get_text_or_none('/html/body/main/div/div/section[3]/div/div[1]/div[2]/div')
        if description:
            description = description.replace('\n더보기', '').strip()

        # 행사 내용 (HTML 처리 포함)
        try:
            html = driver.find_element("xpath", '/html/body/main/div/div/section[3]/div/div[1]/div[3]/p').get_attribute('outerHTML')
            soup = BeautifulSoup(html, 'html.parser')
            for br in soup.find_all("br"):
                br.replace_with("\n")
            contents = soup.get_text(strip=True).replace("[행사내용]", "").strip()
        except:
            contents = None

        homepage = get_attr_or_none('/html/body/main/div/div/section[3]/div/div[2]/div/div[2]/a', 'href')
        poster = get_attr_or_none('/html/body/main/div/div/section[3]/div/div[2]/div/div[1]/a/img', 'src')


        details.append({
            "name": name,
            "location": location,
            "start_date": start_date,
            "end_date": end_date,
            "description": description,
            "contents": contents,
            "homepage": url,
            "poster": poster,
        })

    except Exception as e:
        print(f"[{idx}] 실패: {e}")
        continue

0 단양 소백산 철쭉제
1 밀양아리랑대축제
2 원주용수골꽃양귀비축제
3 공공한옥 밤마실
4 울산대공원 장미축제
5 곡성세계장미축제
6 와일드오차드 티 페스티벌
7 장항항 수산물 꼴갑축제
8 중랑 서울장미축제
9 하동 북천 꽃양귀비 축제
10 여름꽃 & 능소화축제
11 제주푸드앤와인페스티벌
12 공주 페스티벌
13 몽생이 5월 대모험
14 양평메기수염축제
15 포천 한탄강 가든페스타
16 9.81파크 <그래비티 퀘스트 제주>
17 DDP 봄축제 : 디자인 테마파크
18 보롬왓 대파축제
19 옛 전통 봄꽃으로 물들다
20 유럽동화나라축제
21 청도 프로방스 크리스마스 산타마을 빛축제
22 허브아일랜드 라데봄축제
23 라온페스타
24 봄꽃페스타
25 서울거리공연 구석구석 라이브
26 휴애리 봄 수국축제
27 경복궁 생과방
28 부산국제매직페스티벌
29 창덕궁 달빛기행
30 홍성남당항 바다송어 전국 요리대회 및 수산물 소비촉진 행사
31 덕수궁 밤의 석조전
32 생거진천 농다리축제
33 태안봄꽃정원
34 용인 자작나무숲 봄꽃축제
35 청춘마이크 전라제주 청춘퍼레이드
36 양재 아트 살롱
37 천리포수목원 봄축제
38 문화가 있는 날 실버마이크 <전라·제주 권역>
39 문화가 있는 날 청춘마이크 수도강원권
40 문화가 있는 날 청춘마이크 충청권
41 네이처파크 플라워페스티벌
42 피나클랜드 백만송이 튤립 수선화 봄꽃대향연
43 양평모두봄날
44 금남로 차 없는 거리 '걷자잉'
45 구팔일 댕댕이 대잔치!
46 낭만등불축제
47 청도 프로방스 빛축제
48 별빛이 흐르는 정원
49 태안 빛축제
50 남동 빛의 거리
51 K-푸드 페스티벌 넉넉
52 고창청농원 라벤더 축제
53 괴산빨간맛페스티벌
54 글로벌 영도커피페스티벌
55 목포뮤직플레이
56 삼척 장미축제
57 정동야행
58 Colorful Garden 자라섬 꽃 페스타
59 강서 아이들 까치까치 페스티벌
60 강서구 다문화축제
61 무등울림축제
62 베어트리파크 로맨틱 로즈 가든 파티
63 부산세계시민축제
64 서울

In [88]:
# pandas DataFrame으로 변환
detail_df = pd.DataFrame(details)

# 오늘 날짜로 저장
today = datetime.date.today().strftime("%Y-%m-%d")
file_name = f"/home/ys0660/25-1/Demo/data/details_{today}.csv"
detail_df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"✅ 총 {len(detail_df)}개 저장 완료 → {file_name}")

✅ 총 1239개 저장 완료 → /home/ys0660/25-1/Demo/data/details_2025-05-22.csv


In [89]:
detail_df.head()

,name,location,start_date,end_date,description,contents,homepage,poster
0,단양 소백산 철쭉제,충청북도 단양군 단양읍 수변로 38,2025.05.22,2025.05.25,단양 소백산철쭉제는 매 년 5월에 개최된다. 5월 소백산 정상에 핀 철쭉을 즐기며 ...,"1.공연프로그램: 불꽃을 활용한 레이져쇼,동춘서커스, 개막콘서트(이찬원, 클라씨,왁...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
1,밀양아리랑대축제,경상남도 밀양시 삼문동 1-1,2025.05.22,2025.05.25,밀양아리랑대축제는 영남루 대보수사업과 함께 시작된 축제로 현재는 밀양아리랑의 현대적...,1. 대표프로그램- 2025 밀양강오딧세이 : 밀양의 최고 절경인 영남루와 밀양강을...,https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
2,원주용수골꽃양귀비축제,강원특별자치도 원주시 용수골길 311 꽃양귀비축제장,2025.05.22,2025.06.08,"자연경관이 뛰어난 마을에 매료되어, 귀농한 주민이 꽃을 너무 좋아해서 2005년 3...","대표프로그램1. 차분한 꽃 관람2. 체험 프로그램 : 꽃양귀비티셔츠 체험, 깡통열차...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
3,공공한옥 밤마실,서울특별시 종로구 계동길 37 (계동) 북촌문화센터,2025.05.21,2025.05.31,"도심 속 한옥에 드리운 청명한 기운을 느낄 수 있는 초하(初夏)의 계절, 북촌문화센...","1. 전시: 빛 스민 집 展(강현지, 정성아), 밤을 지키는 불빛 展(이승정)2. ...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
4,울산대공원 장미축제,울산광역시 남구 남부순환도로 377 (옥동),2025.05.21,2025.05.25,울산대공원 장미원에서 개최되는 울산대공원 장미축제는 국내 최고의 장미원에서 피어나는...,"1. 메인프로그램 : 개막식(퍼레이드, 개막 점등식, 불꽃쇼, 개막공연)2. 공연프...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
